<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/2_3_PyTorchAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 3: Reverse Mode Automatic Differentiation with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

PyTorch implements Dynamic Reverse Mode Automatic Differentiation, much like we did in the previous exercise. There is one really major difference in what PyTorch provides over our simple example: it works directly with matrices (`Tensor`s) rather than with scalars (although obviously a matrix can represent a scalar).

In this tutorial, we'll explore PyTorch's AD implementation. Note that we're using the API of PyTorch 0.4 or later which simplifies use of AD (previous versions required wrapping all `Tensor`s that you wanted to compute gradients of in `Variable` objects; PyTorch 0.4 removes the need to do this and allows `Tensor`s themselves to track gradients).

We'll start with the simple example we tried earlier in the code block below:

__Task:__ Run the following code and verify the solution is correct

In [ ]:
import torch

# set up the problem
x = torch.tensor(0.5, requires_grad=True)
y = torch.tensor(4.2, requires_grad=True)
z = x * y + torch.sin(x)

print("z = " + str(z.item()))

z.backward() # this goes through the computation graph and accumulates the gradients in the cached .grad attributes
print("dz/dx = " + str(x.grad.item()))
print("dz/dy = " + str(y.grad.item()))

z = 2.57942533493042
dz/dx = 5.077582359313965
dz/dy = 0.5


As with our own AD implementation, PyTorch lets us differentiate through an algorithm.

__Task__: Use the block below to compute the gradient $\partial z/\partial x$ of the following pseudocode algorithm and store the result in the `dzdx` variable:

    x = 0.5
    z = 1
    i = 0
    while i<2:
        z = (z + i) * x * x
        i = i + 1

In [ ]:
dzdx = None

# YOUR CODE HERE
x = torch.tensor(0.5, requires_grad=True)
z = torch.tensor(1.0, requires_grad=True)
for i in range(0,2):
    z = (z + torch.tensor(float(i), requires_grad=True)) * x * x
z.backward()
dzdx = str(x.grad.item())
print(dzdx)

1.5


In [ ]:
a = torch.tensor([[1.,2.],[5,6]], requires_grad=True)
b = torch.tensor([3.,4.], requires_grad=True)

z = torch.mean(a@b)
print(z)
z.backward()

print(a.grad)

tensor(25., grad_fn=<MeanBackward0>)
tensor([[1.5000, 2.0000],
        [1.5000, 2.0000]])


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple


def sgd_factorise(A: torch.Tensor, rank: int, num_epochs=1000, lr=0.001):
    U, V = torch.rand(A.shape[0], rank), torch.rand(A.shape[1], rank)
    for epoch in range(0, num_epochs):
      for r in range(0, A.shape[0]):
          for c in range(0, A.shape[1]):
              e = A[r][c] - U[r] @ V[c].t()
              U[r] = U[r] + lr * e * V[c]
              V[c] = V[c] + lr * e * U[r]
      # print(torch.nn.functional.mse_loss(A, U@V.t(), reduction='sum'))
    return U, V

if __name__ == '__main__':
    basic_array = [[0.3374, 0.6005, 0.1735], [3.3359, 0.0492, 1.8374], [2.9407, 0.5301, 2.02620]]
    basic_tensor = torch.Tensor(basic_array)
    # print("A\n", basic_tensor)
    U, V = sgd_factorise(basic_tensor, 2)
    # print("U_hat\n", U)
    # print("V_hat\n", V)
    # A_hat = U @ V.t()
    # print("A_hat\n", A_hat)
    # loss = torch.nn.functional.mse_loss(reduction='sum', input=basic_tensor, target=A_hat)
    # print(loss)

## PyTorch limitations: in-place operations and aliasing

PyTorch will throw an error at runtime if you try to differentiate through an in-place operation on a tensor. 

__Task__: Run the following code to see this in action.

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y = x.tanh()
y = y + 3
# y.add_(torch.tensor(3.0, requires_grad=True)) # inplace addition
y.backward()

Aliasing is also something that can't be differentiated through and will result in a slightly more cryptic error.

__Task__: Run the following code to see this in action. If you don't understand what this code does add some `print` statements to show the values of `x` and `y` at various points.

In [ ]:
x = torch.tensor([1, 2, 3, 4], requires_grad=True, dtype=torch.float)
y = x[:1]
y.add(3)
y.backward()

## Dealing with multiple outputs

PyTorch can deal with the case where there are multiple output variables if we can formulate the expression in terms of tensor operations. Consider the example from the presentation for example:

$$\begin{cases}
     z = 2x + \sin x\\
     v = 4x + \cos x
\end{cases}$$

We could formulate this as:

$$
\begin{bmatrix}z \\ v\end{bmatrix} = \begin{bmatrix}2 \\ 4\end{bmatrix} \odot \bar{x} + \begin{bmatrix}1 \\ 0\end{bmatrix} \odot \sin\bar x + \begin{bmatrix}0 \\ 1\end{bmatrix} \odot \cos\bar x
$$

where 

$$
\bar x = \begin{bmatrix}x \\ x\end{bmatrix}
$$

and $\odot$ represents the Hadamard or element-wise product. This is demonstrated using PyTorch in the following code block.

__Task:__ run the code below.

In [ ]:
x = torch.tensor([[1.0],[1.0]], requires_grad=True)

zv = ( torch.tensor([[2.0],[4.0]]) * x +
         torch.tensor([[1.0], [0.0]]) * torch.sin(x) +
         torch.tensor([[0.0], [1.0]]) * torch.cos(x) )
        
zv.backward(torch.tensor([[1.0],[1.0]])) # Note as we have "multiple outputs" we must pass in a tensor of weights of the correct size

print(x.grad)

tensor([[2.5403],
        [3.1585]])


__Task:__ Use the following box to write down the derivative of the expression for $\begin{bmatrix}z \\ v\end{bmatrix}$ and verify the gradients $\partial z/\partial x$ and $\partial v/\partial x$ are correct for $x=1$.

YOUR ANSWER HERE

## Gradient descent & gradients with respect to a vector
Let's put everything together and using automatically computed gradients to find the minima of a function by taking steps down the gradient from an initial position. Rather than explicitly creating each input variable as a scalar as in the previous examples, we'll use a vector instead (so our gradients will be with respect to each element of the vector).

__Task:__ work through the following example to see how taking gradients with respect to a vector works & how simple gradient descent optimisation can be implemented.

In [ ]:
# This is our starting vector
initial=[[2.0], [1.0], [10.0]]

# This is the function we will optimise (feel free to work out the analytic minima!)
def function(x):
    return x[0]**2 + x[1]**2 + x[2]**2

x = torch.tensor(initial, requires_grad=True, dtype=torch.float)
for i in range(0,1):
    # manually dispose of the gradient (in reality it would be better to detach and zero it to reuse memory)
    x.grad=None
    # evaluate the function
    J = function(x) 
    # auto-compute the gradients at the previously evaluated point x
    J.backward()
    # compute the update
    z = x - x.grad*0.03 
    x.data = z
    
    if i%10 == 0:
        print((x, function(x).item()))


(tensor([[1.8800],
        [0.9400],
        [9.4000]], requires_grad=True), 92.77799224853516)


__Task__: Answer the following question in the box below: Why must the update in the code above be assigned to a different variable (the `z`) before being assigned to the `data` value of `x`?

YOUR ANSWER HERE

## Differentiating through random operations

We'll end with an example that will be important later in the course: differentiating with respect to the parameters of a random number generator.

Assume that as some part of a differentiable program that we write we wish to incorporate a random element where we sample values, $z$ from a Normal distribution: $z \sim \mathcal{N}(\mu,\sigma^2)$. We want to learn the parameters of the generator $\mu$ and $\sigma^2$, but how can we do this? In a differentiable program setting we want to differentiate with respect to these parameters, but at first glance it isn't at all obvious what this means as the generator _just_ produces numbers which themselves don't have gradients.

The answer is often called the _reparameterisation trick_: If we note that sampling a Normal distribution is equivalent to drawing numbers from a Uniform distribution and scaling and shifting them: $z \sim \mathcal{N}(\mu,\sigma^2) \equiv z \sim \mu + \sigma\mathcal{U}(0,1)\equiv z = \mu + \sigma \zeta\, \rm{where}\, \zeta\sim\mathcal{U}(0,1)$. With this reparameterisation the gradients with respect to the parameters are obvious.

The following code block demonstrates this in practice; each of the gradients can be interpreted as how much an infintesimal change in $\mu$ or $\sigma$ would change $z$ if we could repeat the sampling operation again with the same value of `torch.rand(1)` being produced:

In [ ]:
mu = torch.tensor(1.0, requires_grad=True)
sigma = torch.tensor(1.0, requires_grad=True)

for i in range(10):
    mu.grad = None
    sigma.grad = None
    
    z = mu + sigma * torch.rand(1)
    z.backward()
    print("z:", z.item(), "\tdzdmu:", mu.grad.item(), "\tdzdsigma:", sigma.grad.item())

z: 1.1395483016967773 	dzdmu: 1.0 	dzdsigma: 0.13954836130142212
z: 1.2383465766906738 	dzdmu: 1.0 	dzdsigma: 0.23834657669067383
z: 1.3980185985565186 	dzdmu: 1.0 	dzdsigma: 0.39801865816116333
z: 1.280860424041748 	dzdmu: 1.0 	dzdsigma: 0.28086036443710327
z: 1.5323940515518188 	dzdmu: 1.0 	dzdsigma: 0.5323940515518188
z: 1.5215588808059692 	dzdmu: 1.0 	dzdsigma: 0.5215588808059692
z: 1.9777328968048096 	dzdmu: 1.0 	dzdsigma: 0.9777329564094543
z: 1.2924656867980957 	dzdmu: 1.0 	dzdsigma: 0.2924656867980957
z: 1.058884620666504 	dzdmu: 1.0 	dzdsigma: 0.05888468027114868
z: 1.2458056211471558 	dzdmu: 1.0 	dzdsigma: 0.24580562114715576
